In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy

import torch
import socialforce

# Fit 1D

The preferred speed needs to be varied. Otherwise the symmetry of the problem creates unrealistic scenarios where the two pedestrians get stuck.

{ref}`Scenarios <scenarios>`.

In [ ]:
circle = socialforce.scenarios.Circle()
scenarios_1 = circle.generate(1)
scenarios_5 = circle.generate(5)
scenarios_20 = circle.generate(20)

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas(ncols=3, figsize=(12, 4)) as (ax1, ax2, ax3):
    socialforce.show.states(ax1, scenarios_1[0])

    socialforce.show.states(ax2, scenarios_5[-1])
    for scene in scenarios_5[:-1]:
        socialforce.show.states(ax2, scene, alpha=0.1)

    socialforce.show.states(ax3, scenarios_20[-1])
    for scene in scenarios_20[:-1]:
        socialforce.show.states(ax3, scene, alpha=0.1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_1)
V = socialforce.PedPedPotentialMLP().double()
initial_state_dict = copy.deepcopy(V.state_dict())

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=10.0)
socialforce.Trainer(simulator_factory, opt, true_experience).loop(50, log_interval=10)
final_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE CODE
with socialforce.show.canvas(ncols=2) as (ax1, ax2):
    socialforce.show.potential1D_parametric(
        circle.ped_ped, ax1, ax2, 
        label=r'true $V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='gray')

    V.load_state_dict(initial_state_dict)
    socialforce.show.potential1D(V, ax1, ax2, label=r'initial MLP($b$)', linestyle='dashed', color='C0')

    V.load_state_dict(final_state_dict)
    socialforce.show.potential1D(V, ax1, ax2, label=r'MLP($b$)', color='C0')